In [ ]:
import torch
from torch import nn
import utils.data_utils as data_utils
from models.FenceGAN import Generator, Discriminator
from training.FenceGAN_train import train_kdd99, train_financial
import matplotlib.pyplot as plt

In [ ]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(DEVICE)

# Hyperparameters

In [ ]:
# choose dataset (kdd99, aapl, gm, axp)
dataset = "aapl"

In [ ]:
# Data
batch_size = 8
random_seed = 0
num_features = 7
seq_length = 30
seq_stride = 10
gen_seq_len = seq_length
# Model
latent_dim = 30
# Training
gen_lr = 1e-4
gen_wd = 1e-3
dis_lr = 8e-6
dis_wd = 1e-3
dis_momentum = 0.9
num_epochs = 50

# Dataset

In [ ]:
if dataset == "kdd99":
    train_dl, test_dl = data_utils.kdd99(seq_length, seq_stride, num_features, gen_seq_len, batch_size)
else:
    file_path = 'data/financial_data/Stocks/'+dataset+'.us.txt'
    tscv_dl_list = data_utils.load_stock_as_crossvalidated_timeseries(file_path, seq_length, seq_stride, gen_seq_len, batch_size, normalise=True)

# Model 

In [ ]:
# use xavier initialization for weights
def init_weights(m):
    if type(m) == nn.Linear:
        torch.nn.init.xavier_uniform_(m.weight)   

In [ ]:
generator = Generator(input_dim=latent_dim,output_dim=num_features).to(device=DEVICE)
generator.apply(init_weights)

In [ ]:
discriminator = Discriminator(input_dim=num_features).to(device=DEVICE)
discriminator.apply(init_weights)

# Loss & Optimizer

In [ ]:
generator_optim = torch.optim.Adam(generator.parameters(), lr=gen_lr, weight_decay=gen_wd)
discriminator_optim = torch.optim.SGD(discriminator.parameters(), lr=dis_lr, weight_decay=dis_wd)

In [ ]:
def dispersion_loss(G_out, y_pred, y_true):
    dispersion_weight = 30
    loss_b = nn.BCELoss()(y_pred, y_true)
    center = G_out.mean(dim=0, keepdims=True)
    distance_xy = torch.square(torch.subtract(G_out, center))
    if G_out.dim() > 1:
        distance = distance_xy.sum(dim=1)
    else:
        distance = distance_xy.sum()
    avg_distance = torch.sqrt(distance).mean()
    loss_d = torch.reciprocal(avg_distance)
    loss = loss_b + dispersion_weight*loss_d
    return loss

In [ ]:
def disc_loss(real_pred, real_true, fake_pred, fake_true):
    gen_weight = 0.5
    loss_real = nn.BCELoss()(real_pred, real_true)
    loss_gen = nn.BCELoss()(fake_pred, fake_true)
    loss = loss_real + gen_weight * loss_gen
    return loss

# Training & Evaluation

In [ ]:
if dataset == "kdd99":
    train_kdd99(seq_length, latent_dim,  train_dl, test_dl, discriminator, generator, discriminator_optim, generator_optim, disc_loss, dispersion_loss, random_seed, num_epochs, DEVICE)
else:
    train_financial(seq_length, latent_dim, tscv_dl_list, discriminator, generator, discriminator_optim, generator_optim, disc_loss, dispersion_loss, random_seed, num_epochs, DEVICE)

# Generated Samples

In [ ]:
from training.FenceGAN_train import sample_Z
latent_sample = sample_Z(batch_size, seq_length, latent_dim)
if dataset == "kdd99":
    batch = next(iter(test_dl))
else:
    batch = next(iter(tscv_dl_list[4][1]))
gen_sample = generator(latent_sample.to(DEVICE)).cpu().detach()[0]
x = batch[0][5]

In [ ]:
# generated Sample
plt.plot(gen_sample)

In [ ]:
# Real Sample
plt.plot(x)